## Notebook Setup

In [1]:
import os
import json
import yaml

from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
with open('../keys_stream/my_api_key.yaml', 'r') as f:
    api_keys = yaml.safe_load(f)

In [42]:
chat_model = ChatOpenAI(api_key = api_keys['OPENAI_API_KEY'],
                        model_name = 'gpt-4o')

In [4]:
STATE_GENERATION_PROMPT = '''Generate a list of the 50 U.S. states in alphabetical order, separated by commas. Do not include any extra text nor punctuation. The output should be a simple list of state names.'''

output_parser = CommaSeparatedListOutputParser()

state_generation_prompt_template = PromptTemplate(
    template = STATE_GENERATION_PROMPT,
    partial_variables = {'format_instructions': output_parser.get_format_instructions()},
)

state_generation_chain = state_generation_prompt_template | chat_model | output_parser

In [5]:
if not os.path.exists('state_results.json'):
    state_results = state_generation_chain.invoke(input = {})

    with open('state_results.json', 'w') as f:
        json.dump(state_results, f)

else:
    with open('state_results.json', 'r') as f:
        state_results = json.load(f)

In [6]:
STATE_BLURB_PROMPT = '''Write a one-page fact sheet about the U.S. state of {state_name}. The tone should be engaging, informative, and suitable for general audiences including students and travelers. Keep each fact brief and clear. Organize the information under the following headings:
	•	Nickname and Motto: Include the state’s nickname and official motto.
	•	Capital and Major Cities: Mention the capital and 2-3 other major cities.
	•	Geography: Describe the landscape, notable natural features, and bordering states.
	•	State Symbols: Include state bird, flower, tree, and animal.
	•	History Highlights: Mention a few key historical events or facts.
	•	Economy: What the state is known for economically (e.g., agriculture, tech, tourism).
	•	Fun Facts: Share 2-3 quirky or interesting trivia points.

Keep the total output under 300 words, written in bullet-point or brief paragraph format.'''

state_blurb_prompt_template = PromptTemplate(
    template = STATE_BLURB_PROMPT,
    input_variables = ['state_name'],
)

state_blurb_chain = state_blurb_prompt_template | chat_model

In [10]:
# for state_name in state_results:
#     if not os.path.exists(f'state_blurbs/{state_name}.md'):

#         state_blurb = state_blurb_chain.invoke(input = {'state_name': state_name})

#         with open(f'state_blurbs/{state_name}.md', 'w') as f:
#             f.write(state_blurb.content)

#     print(f"State: {state_name}")

In [11]:
chat_model_json = chat_model.bind(response_format={"type": "json_object"})

In [43]:
STATE_TRIVIA_QA_PROMPT = '''You are a trivia expert generating fun and educational trivia questions about U.S. states.

Your task is to generate a list of 5 trivia question-answer pairs about the state of {{state_name}}. 
Each question should be unique, factual, and relevant to the state’s geography, history, culture, symbols, or notable facts.
All questions should only be about the state of {{state_name}}. Do not provide questions about other states or general U.S. trivia.

Output the results as a JSON list, where each item is a dictionary with the keys:
- "question": the trivia question
- "answer": the correct answer

Only output the raw JSON. Do not include any explanations, markdown, or text outside the JSON. Only include the trivia questions and answers.

Example format:
[
  {{
    "question": "What is the capital city of Illinois?",
    "answer": "Springfield"
  }},
  {{
    "question": "Which famous U.S. president was born in Kentucky but built his political career in Illinois?",
    "answer": "Abraham Lincoln"
  }},
  ...
]
'''
state_triva_qa_prompt_template = PromptTemplate(
    template = STATE_TRIVIA_QA_PROMPT,
    input_variables = ['state_name'],
)

state_trivia_qa_chain = state_triva_qa_prompt_template | chat_model

In [38]:
response = state_trivia_qa_chain.invoke(input = {'state_name': 'California'})

In [39]:
json.loads(response.content)

[{'question': 'What is the capital city of Texas?', 'answer': 'Austin'},
 {'question': 'Which Texas city is known as the live music capital of the world?',
  'answer': 'Austin'},
 {'question': 'What iconic structure in Texas is known for its unique design and serves as a symbol of the Lone Star State?',
  'answer': 'The Alamo'},
 {'question': "Which Texas custom is celebrated annually with a festival honoring the state's cowboy culture?",
  'answer': 'The Houston Livestock Show and Rodeo'},
 {'question': 'What natural wonder located in Texas is the largest hill country in the United States?',
  'answer': 'The Edwards Plateau'}]

In [40]:
response_json['trivia']

[{'question': 'What is the capital city of Maine?', 'answer': 'Augusta'},
 {'question': "Which U.S. state is known as the 'Pine Tree State'?",
  'answer': 'Maine'},
 {'question': 'What is the signature dish of Maine, often associated with its coastal cuisine?',
  'answer': 'Lobster'},
 {'question': 'Which national park, located in Maine, is known for its stunning coastal views and hiking trails?',
  'answer': 'Acadia National Park'},
 {'question': 'What is the name of the famous lighthouse located in Cape Elizabeth, Maine?',
  'answer': 'Portland Head Light'}]

In [45]:
for state_name in state_results:
    print(f'Current state: {state_name}')
    if not os.path.exists(f'state_trivia/{state_name}.json'):

        state_trivia = state_trivia_qa_chain.invoke(input = {'state_name': state_name})

        state_trivia_json = json.loads(state_trivia.content)

        with open(f'state_trivia/{state_name}.json', 'w') as f:
            json.dump(state_trivia_json, f)

Current state: Alabama
Current state: Alaska
Current state: Arizona
Current state: Arkansas
Current state: California
Current state: Colorado
Current state: Connecticut
Current state: Delaware
Current state: Florida
Current state: Georgia
Current state: Hawaii
Current state: Idaho
Current state: Illinois
Current state: Indiana
Current state: Iowa
Current state: Kansas
Current state: Kentucky
Current state: Louisiana
Current state: Maine
Current state: Maryland
Current state: Massachusetts
Current state: Michigan
Current state: Minnesota
Current state: Mississippi
Current state: Missouri
Current state: Montana
Current state: Nebraska


KeyboardInterrupt: 